In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
file_path = "heijden.csv"

# Data cleaning
def clean_raw_data(file_path):
    # Read in the data
    raw_data = pd.read_csv(file_path, low_memory=False)
    columns_keep = ["tic", "sic", "naics", "gsubind", "at", "che", "rect", "invt", "act",
                    "ppent", "ivaeq", "intan", "ap", "dlc", "ui", "lct", "dltt", "lt",
                    "pstk", "ceq", "sale", "cogs", "xrd", "ib"]
    raw_data = raw_data[columns_keep]
    raw_shape1 = raw_data.shape
    print(f'The size of raw_data is {raw_shape1}.')

    # Drop rows with total assets not available
    raw_data.dropna(subset=["at"], inplace=True)
    raw_shape2 = raw_data.shape
    print(f'The # of obs. dropped because of unavailable total assets is {raw_shape1[0] - raw_shape2[0]}.')

    # Drop rows with total assets not positive
    index_at_drop = raw_data[raw_data["at"] <= 0].index.tolist()
    raw_data.drop(index=index_at_drop, inplace=True)
    raw_shape3 = raw_data.shape
    print(f'The # of obs. dropped because of non_positive total assets is {raw_shape2[0] - raw_shape3[0]}.')

    # Drop rows without vaild classification codes
    raw_data.dropna(subset=["sic", "naics", "gsubind"], inplace=True)
    

    # Clean the balance sheet
    def balance_sheet_clean(balance_sheet):
        index_set = set()
        for col in balance_sheet.columns:
            index1 = balance_sheet[balance_sheet[col] >= raw_data["at"]].index.tolist()
            index2 = balance_sheet[balance_sheet[col] < 0].index.tolist()
            index = pd.Series(index1 + index2).unique()
        return index

    balance_sheet = raw_data.iloc[:, 5:]
    index = balance_sheet_clean(balance_sheet)

    raw_data.drop(index=index, inplace=True)

    print(f'The # of obs. dropped because of unusual items is {len(index)}.')
    print(f'The # of companies is {raw_data["tic"].nunique()}.')

    return raw_data

# Read and clean raw data
raw_data = clean_raw_data(file_path)


The size of raw_data is (126556, 24).
The # of obs. dropped because of unavailable total assets is 28392.
The # of obs. dropped because of non_positive total assets is 423.
The # of obs. dropped because of unusual items is 40052.
The # of companies is 8863.


In [3]:
# Data engineering
data = pd.DataFrame({
    # Identification and classifications
    "Ticker" : raw_data["tic"],
    "SIC" : raw_data["sic"].apply(str),
    "NAICS" : raw_data["naics"].apply(str),
    "GICS" : raw_data["gsubind"].apply(str),
    
    # Compute common size components
    ## Assets
    "A1" : raw_data["che"].fillna(0) / raw_data["at"],   # Cash & Marketable Securities
    "A2" : raw_data["rect"].fillna(0) / raw_data["at"],  # Receivables
    "A3" : raw_data["invt"].fillna(0) / raw_data["at"],  # Inventories
    "A4" : raw_data["act"].fillna(0) / raw_data["at"],   # Total current assets
    "A5" : raw_data["ppent"].fillna(0) / raw_data["at"], # Net Plant & Equipment
    "A6" : raw_data["ivaeq"].fillna(0) / raw_data["at"], # Investments
    "A7" : raw_data["intan"].fillna(0) / raw_data["at"], # Goodwill & Intangibles

    ## Liabilities
    "L1" : raw_data["ap"].fillna(0) / raw_data["at"],   # Accounts Payable
    "L2" : raw_data["dlc"].fillna(0) / raw_data["at"],  # Total debt in current liabilities
    "L3" : raw_data["ui"].fillna(0) / raw_data["at"],   # Unearned Revenues
    "L4" : raw_data["lct"].fillna(0) / raw_data["at"],  # Total current liabilities
    "L5" : raw_data["dltt"].fillna(0) / raw_data["at"], # total long-term debt
    "L6" : raw_data["lt"].fillna(0) / raw_data["at"],   # Total liabilities

    ## Equity
    "E1" : raw_data["pstk"].fillna(0) / raw_data["at"], # Preferred stock
    "E2" : raw_data["ceq"].fillna(0) / raw_data["at"],  # Common stock

    ## Ratios
    "R1" : (raw_data["sale"] - raw_data["cogs"]) / raw_data["sale"],                   # Gross Margin
    "R2" : raw_data["xrd"] / raw_data["sale"],                                         # R&D/Sales
    "R3" : raw_data["ib"] / raw_data["sale"],                                          # Net Income/Sales
    "R4" : raw_data["rect"] * (raw_data["sale"] / 365),                                # Days of Receivables
    "R5" : raw_data["cogs"] / raw_data["invt"],                                        # Inventory Turnover
    "R6" : raw_data["sale"] / raw_data["ppent"],                                       # Fixed Asset Turnover
    "R7" : raw_data["sale"] / raw_data["at"],                                          # Total Asset Turnover
    "R8" : raw_data["ib"] /  raw_data["at"],                                           # Net Income/Assets
    "R9" : raw_data["ib"] / raw_data["ceq"],                                           # Net Income/Equity
    "R10" : raw_data["at"] / raw_data["ceq"],                                          # Assets/Equity
    "R11" : (raw_data["dlc"] + raw_data["dltt"]) / raw_data["ceq"],                    # Debt/Equity
    "R12" : raw_data["dltt"] / (raw_data["pstk"] + raw_data["ceq"] + raw_data["dltt"]) # L/T Debt/Total Capital
    })

# Reset index
data.reset_index(inplace=True, drop=True)

data_shape1 = data.shape
print(f'The size of data is {data_shape1}.')

The size of data is (56463, 31).


In [4]:
# Extract levels of classes
def split_gics(codes):
    gics_l1 = []
    gics_l2 = []
    gics_l3 = []
    gics_l4 = []

    for code in codes:
        gics_l1.append(code[:2])
        gics_l2.append(code[2:4])
        gics_l3.append(code[4:6])
        gics_l4.append(code[6:8])

    output = pd.DataFrame({
        "GICS L1": gics_l1,
        "GICS L2": gics_l2,
        "GICS L3": gics_l3,
        "GICS L4": gics_l4
    })
    
    return output

def split_sic(codes):
    sic_l1 = []
    sic_l2 = []
    sic_l3 = []

    for code in codes:
        sic_l1.append(code[:2])
        sic_l2.append(code[2:3])
        sic_l3.append(code[3:4])

    output = pd.DataFrame({
        "SIC L1": sic_l1,
        "SIC L2": sic_l2,
        "SIC L3": sic_l3,
    })
    
    return output

def split_naics(codes):
    naics_l1 = []
    naics_l2 = []
    naics_l3 = []
    naics_l4 = []
    naics_l5 = []

    for code in codes:
        naics_l1.append(code[:2])
        naics_l2.append(code[2:3])
        naics_l3.append(code[3:4])
        naics_l4.append(code[4:5])
        naics_l5.append(code[5:6])

    output = pd.DataFrame({
        "NAICS L1": naics_l1,
        "NAICS L2": naics_l2,
        "NAICS L3": naics_l3,
        "NAICS L4": naics_l4,
        "NAICS L5": naics_l5
    })
    
    return output

h_sic = split_sic(data["SIC"])
h_gics = split_gics(data["GICS"])
h_naics = split_naics(data["NAICS"])

data2 = pd.concat([h_gics, h_naics, h_sic, data], axis=1) # we need to drop ["SIC", "GICS", "NAICS", "Ticker"] before split



In [5]:
# Missing values
missing_values = data.isna().sum()
missing_values = missing_values[missing_values > 0]

print("Missing values:")
for column, missing_count in missing_values.items():
    print(f" '{column}' has {missing_count} missing values.")

# Infinite values
inf_values = (data.iloc[:, 4:] == np.inf).sum()
inf_values = inf_values[inf_values > 0]

print("Infinite values:")
for column, inf_count in inf_values.items():
    print(f" '{column}' has {inf_count} infinite values." )

# Empty strings
emp_values = data2.apply(lambda x: x == "").sum()
emp_values = emp_values[emp_values > 0]

print("Empty strings")
for column, emp_count in emp_values.items():
    print(f" '{column}' has {emp_count} empty strings." )

Missing values:
 'Ticker' has 25 missing values.
 'R1' has 10858 missing values.
 'R2' has 37939 missing values.
 'R3' has 10487 missing values.
 'R4' has 10772 missing values.
 'R5' has 11609 missing values.
 'R6' has 12333 missing values.
 'R7' has 10460 missing values.
 'R8' has 298 missing values.
 'R9' has 451 missing values.
 'R10' has 161 missing values.
 'R11' has 732 missing values.
 'R12' has 429 missing values.
Infinite values:
 'R1' has 11 infinite values.
 'R2' has 21 infinite values.
 'R3' has 453 infinite values.
 'R5' has 11832 infinite values.
 'R6' has 1540 infinite values.
 'R9' has 7 infinite values.
 'R10' has 27 infinite values.
 'R11' has 23 infinite values.
Empty strings
 'NAICS L4' has 27 empty strings.
 'NAICS L5' has 1146 empty strings.
 'SIC L3' has 181 empty strings.


In [6]:
# Check length of codes
print(data["SIC"].apply(len).unique())
print(data["GICS"].apply(len).unique())
print(data["NAICS"].apply(len).unique())

[4 3]
[10]
[8 6 7 5 4]


In [7]:
heijden_data = pd.concat([data2["NAICS L1"], data2.iloc[:, 16:]], axis=1)

In [8]:
np.round(heijden_data.groupby("NAICS L1").median().T, 2)

NAICS L1,11,21,22,23,31,32,33,42,44,45,48,49,51,52,53,54,56,61,62,71,72,81,99
A1,0.07,0.06,0.01,0.09,0.07,0.10,0.14,0.04,0.06,0.11,0.04,0.11,0.16,0.02,0.01,0.13,0.09,0.35,0.05,0.07,0.07,0.03,0.12
A2,0.03,0.04,0.04,0.10,0.11,0.11,0.15,0.22,0.04,0.03,0.04,0.18,0.09,0.03,0.01,0.23,0.16,0.07,0.13,0.02,0.03,0.05,0.08
A3,0.08,0.01,0.02,0.18,0.15,0.10,0.13,0.23,0.22,0.24,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.03,0.02
A4,0.33,0.18,0.09,0.32,0.43,0.42,0.55,0.63,0.47,0.52,0.13,0.36,0.36,0.00,0.00,0.50,0.43,0.56,0.27,0.13,0.14,0.20,0.33
A5,0.37,0.71,0.73,0.09,0.21,0.25,0.14,0.11,0.27,0.26,0.68,0.39,0.08,0.01,0.00,0.06,0.08,0.14,0.16,0.50,0.48,0.19,0.11
A6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A7,0.01,0.00,0.01,0.02,0.19,0.11,0.13,0.12,0.09,0.05,0.04,0.11,0.28,0.01,0.01,0.31,0.25,0.10,0.34,0.21,0.10,0.35,0.05
L1,0.02,0.04,0.03,0.06,0.08,0.06,0.07,0.14,0.11,0.13,0.02,0.06,0.03,0.01,0.01,0.05,0.04,0.03,0.03,0.02,0.03,0.03,0.05
L2,0.01,0.00,0.03,0.02,0.02,0.01,0.01,0.01,0.01,0.00,0.02,0.01,0.01,0.02,0.02,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.02
L3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
